# DS320 Final Project
## Think Progress Data Collection Notebook
### Noah B Johnson

## Import Modules

In [64]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import urllib3
import random
import time
import json
from tinydb import TinyDB, Query
import feedparser

## Configure Environment

In [66]:
# feed url for scraping
baseUrl = "https://thinkprogress.org/feed"
# setup json file for tinydb
db = TinyDB('db.json')

## Aggregate Historical RSS Feeds

In [92]:
def aggregate(daysBack):
    # daysBack = How many days of data to collect
    
    # The number of archive pages (days) scraped
    pages = 0
    
    # Set the iterative variables for the url
    day = 1
    month = 6
    year = 2016
    
    # query object for checking for duplicates
    q = Query()
    
    # Go through the archive pages and add articles to the db
    while pages < daysBack:
        # Create archive url based on date iterators
        if day < 10:
            dayPrint = "0" + str(day)
        else:
            dayPrint = str(day)
        if month < 10:
            monthPrint = "0" + str(month)
        else:
            monthPrint = str(month)
        feedLink = "https://web.archive.org/web/{}{}{}/{}".format(year,monthPrint,dayPrint,baseUrl)
        # Get the feed object entries
        entries = feedparser.parse(feedLink)['entries']
        if len(entries) > 0:
            print(entries[0]['published'])
            for entry in entries:
                if not db.contains(q.url == entry['link']):
#                     print(entry)
                    db.insert({
                        'url': entry['link'],
                        'time': entry['published'],
                        'author': entry['author'],
                        'title': entry['title'],
                        'body': BeautifulSoup(entry['summary_detail']['value'], 'html.parser').get_text(),
                        'site': 'thinkprogress'
                    })
        
        # Take the date down by one
        day = day - 1
        if day == 0:
            month = month - 1
            day = 31
        if month == 0:
            year = year - 1
            month = 12
            
        # Update the pages scraped value
        pages +=1

In [97]:
# Run the aggregation function
# Loads the feed entries into the database
# urlList = aggregate(1100)